In [1]:
import networkx as nx
import os
import csv
import gzip
import collections
import re
import io
import json
import xml.etree.ElementTree as ET

import requests
import pandas as pd
import pandas
import numpy as np
import math

In [2]:
G_diseases = nx.read_pajek("../networks/disease_disease_network.net")

In [3]:
G_targets = nx.read_pajek("../networks/targets_targets_network.net")

In [4]:
G_drugs = nx.read_pajek("../networks/drug_drug_network.net")

In [5]:
G_heterogeneous = nx.Graph()

In [6]:
G_heterogeneous.add_nodes_from(G_diseases.nodes())
G_heterogeneous.add_nodes_from(G_targets.nodes())
G_heterogeneous.add_nodes_from(G_drugs.nodes())

G_heterogeneous.number_of_nodes()

5901

In [7]:
G_heterogeneous.add_edges_from(G_diseases.edges())
G_heterogeneous.add_edges_from(G_targets.edges())
G_heterogeneous.add_edges_from(G_drugs.edges())

G_heterogeneous.number_of_edges()

5692147

In [8]:
data = pd.read_csv('../data/msb201126-s1.txt', delimiter = "\t")
data_omim = pd.read_csv('../data_omim/full_omim_table.txt', delimiter = "\t")
drugbank_slim_df = pd.read_table('../data/drugbank-slim.tsv')
drugbank_similarity_df = pd.read_table('../data/similarity-slim.tsv')
drugbank_similarity_df

,compound0,compound1,similarity
0,DB00014,DB00035,0.4902
1,DB00014,DB00050,0.6841
2,DB00014,DB00091,0.1911
3,DB00014,DB00093,0.4336
4,DB00014,DB00104,0.4080
...,...,...,...
1200470,DB09019,DB09028,0.1978
1200471,DB09019,DB07402,0.2182
1200472,DB09020,DB09028,0.1525
1200473,DB09020,DB07402,0.2190


In [382]:
def convert_disease_name_to_desease_number(disease_name):
    for row in (data_omim.loc[data_omim['phenotype'] == disease_name.lower().capitalize()].iterrows()):
        if not math.isnan(row[1]['phenotypeMimNumber']):
            return row[1]['phenotypeMimNumber']
    
def convert_drug_name_to_drug_id(drug_name):
    for row in (drugbank_slim_df.loc[drugbank_slim_df['name'] == drug_name].iterrows()):
        return row[1]['drugbank_id']
    
def convert_drug_id_to_drug_name(drug_id):
    for row in (drugbank_slim_df.loc[drugbank_slim_df['drugbank_id'] == drug_id].iterrows()):
        return row[1]['name']

In [10]:
#dopolni to za drug_disease!!!

count = 0
for row in data.iterrows():
    #if 'Cancer' in row[1]['Disease name']:
    #    print(row[1]['Disease name'])
    #print(row[1]['Disease name'])
    
    disease_number = convert_disease_name_to_desease_number(row[1]['Disease name'])
    drug_id = convert_drug_name_to_drug_id(row[1]['Drug name'])
    
    if disease_number != None and drug_id != None:
        #print(int(disease_number))
        if G_diseases.has_node(str(int(disease_number))) and G_drugs.has_node(drug_id):
            #count += 1
            G_heterogeneous.add_edge(str(int(disease_number)), drug_id, weight=1.0)
        
count

0

In [11]:
G_heterogeneous.number_of_edges()

5692445

In [12]:
convert_disease_name_to_desease_number("Atherosclerosis, susceptibility to")

nan

In [13]:
G_diseases.has_node(str(int(180300.0)))

True

In [14]:
xml_path = os.path.join('../download', 'drugbank.xml.gz')
with gzip.open(xml_path) as xml_file:
    tree = ET.parse(xml_file)
root = tree.getroot()

ns = '{http://www.drugbank.ca}'
inchikey_template = "{ns}calculated-properties/{ns}property[{ns}kind='InChIKey']/{ns}value"
inchi_template = "{ns}calculated-properties/{ns}property[{ns}kind='InChI']/{ns}value"

In [15]:
protein_rows = list()
for i, drug in enumerate(root):
    drugbank_id = drug.findtext(ns + "drugbank-id[@primary='true']")
    for category in ['target', 'enzyme', 'carrier', 'transporter']:
        proteins = drug.findall('{ns}{cat}s/{ns}{cat}'.format(ns=ns, cat=category))
        for protein in proteins:
            row = {'drugbank_id': drugbank_id, 'category': category}
            row['organism'] = protein.findtext('{}organism'.format(ns))
            row['known_action'] = protein.findtext('{}known-action'.format(ns))
            actions = protein.findall('{ns}actions/{ns}action'.format(ns=ns))
            row['actions'] = '|'.join(action.text for action in actions)
            uniprot_ids = [polypep.text for polypep in protein.findall(
                "{ns}polypeptide/{ns}external-identifiers/{ns}external-identifier[{ns}resource='UniProtKB']/{ns}identifier".format(ns=ns))]            
            if len(uniprot_ids) != 1:
                continue
            row['uniprot_id'] = uniprot_ids[0]
            ref_text = protein.findtext("{ns}references[@format='textile']".format(ns=ns))
            pmids = re.findall(r'pubmed/([0-9]+)', ref_text)
            row['pubmed_ids'] = '|'.join(pmids)
            protein_rows.append(row)

protein_df = pandas.DataFrame.from_dict(protein_rows)

In [16]:
count = 0

for node in G_drugs.nodes():
    for row in (protein_df.loc[protein_df['drugbank_id'] == node].iterrows()):
        #print(row[1]['uniprot_id'])
        #G_targets.add_node(row[1]['uniprot_id'])
        uni_prot_id = row[1]['uniprot_id']
        
        #print(uni_prot_id)
        
        if G_targets.has_node(str(uni_prot_id)) and G_drugs.has_node(node):
            #count += 1
            G_heterogeneous.add_edge(str(uni_prot_id), node, weight=1.0)
            
G_heterogeneous.number_of_edges()

5702375

In [17]:
#construct A_R_R, A_R_T, A_R_D, A_T_T, A_T_D, A_D_D

#drugs: G_drugs
rows, cols = (G_drugs.number_of_nodes(), G_drugs.number_of_nodes())

A_R_R = np.matrix([[0]*cols]*rows, dtype=float)

In [18]:
drugs_nodes_list = list(G_drugs.nodes())

for edge in G_drugs.edges():
    node_idx_1 = drugs_nodes_list.index(edge[0])
    node_idx_2 = drugs_nodes_list.index(edge[1])
    weight = G_drugs.get_edge_data(edge[0], edge[1])[0]['weight']
    
    A_R_R[node_idx_1, node_idx_2] = weight

In [19]:
#proteins: G_targets

rows, cols = (G_targets.number_of_nodes(), G_targets.number_of_nodes())

A_T_T = np.matrix([[0]*cols]*rows, dtype=float)

In [20]:
targets_nodes_list = list(G_targets.nodes())

for edge in G_targets.edges():
    node_idx_1 = targets_nodes_list.index(edge[0])
    node_idx_2 = targets_nodes_list.index(edge[1])
    weight = G_targets.get_edge_data(edge[0], edge[1])[0]['weight']
    
    A_T_T[node_idx_1, node_idx_2] = weight

In [21]:
#diseases: G_disease

rows, cols = (G_diseases.number_of_nodes(), G_diseases.number_of_nodes())

A_D_D = np.matrix([[0]*cols]*rows, dtype=float)

In [22]:
diseases_nodes_list = list(G_diseases.nodes())

for edge in G_diseases.edges():
    node_idx_1 = diseases_nodes_list.index(edge[0])
    node_idx_2 = diseases_nodes_list.index(edge[1])
    weight = G_diseases.get_edge_data(edge[0], edge[1])[0]['weight']
    
    A_D_D[node_idx_1, node_idx_2] = weight

In [23]:
#drug_desease: G_drugs and G_diseases

rows, cols = (G_drugs.number_of_nodes(), G_diseases.number_of_nodes())

A_R_D = np.matrix([[0]*cols]*rows, dtype=float)

In [24]:
count = 0

for drug in G_drugs.nodes():
    for disease in G_diseases.nodes():
        if G_heterogeneous.has_edge(drug, disease):
            drug_idx = drugs_nodes_list.index(drug)
            disease_idx = diseases_nodes_list.index(disease)
            A_R_D[drug_idx, disease_idx] = 1
            count += 1
            
count

298

In [25]:
#drug_target: G_drugs and G_targets

rows, cols = (G_drugs.number_of_nodes(), G_targets.number_of_nodes())

A_R_T = np.matrix([[0]*cols]*rows, dtype=float)

In [26]:
count = 0

for drug in G_drugs.nodes():
    for target in G_targets.nodes():
        if G_heterogeneous.has_edge(drug, target):
            drug_idx = drugs_nodes_list.index(drug)
            target_idx = targets_nodes_list.index(target)
            A_R_T[drug_idx, target_idx] = 1
            count += 1

count

9930

In [27]:
#disease_target: G_diseases and G_targets

rows, cols = (G_diseases.number_of_nodes(), G_targets.number_of_nodes())

A_D_T = np.matrix([[0]*cols]*rows, dtype=float)

In [28]:
count = 0

for disease in G_diseases.nodes():
    for target in G_targets.nodes():
        if G_heterogeneous.has_edge(disease, target):
            disease_idx = disease_nodes_list.index(disease)
            target_idx = targets_nodes_list.index(target)
            A_D_T[disease_idx, target_idx] = 1
            count += 1


In [29]:
A_D_R = A_R_D.transpose()
A_T_R = A_R_T.transpose()
A_T_D = A_D_T.transpose()

In [327]:
gama = 0.2

gama_drug = 0.4
gama_disease = 0.4
gama_target = 0.2

gama_drug_disease = 0.4
gama_target_disease = 0.4
gama_target_drug = 0.2

assert gama_drug + gama_disease + gama_target == 1.0

In [328]:
# i - vrstica, j - stolpec
def sum_all_elements_in_row(matrix, i, dictionary):
    cur_sum = 0
    
    if i not in dictionary:
        if i < matrix.shape[0]:
            for j in range(matrix.shape[1]):
                cur_sum += matrix[i, j]
        dictionary[i] = cur_sum
    else:
        cur_sum = dictionary[i]
        
    return cur_sum
    

In [329]:
def get_element(matrix, i, j):
    if i < matrix.shape[0] and j < matrix.shape[1]:
        return matrix[i, j]
    return 0

In [330]:
A_D_D.shape

(2741, 2741)

In [331]:
def create_intra_transition_matrix(A_1_1, A_1_2, A_1_3, gama_1_2, gama_1_3):

    M = np.matrix(np.zeros(A_1_1.shape), dtype=float)
    dict_M = {}
    dict_uniq_rows = {}

    for i in range(M.shape[0]):
        for j in range(M.shape[1]):
            sum_A_1_1 = sum_all_elements_in_row(A_1_1, i, dict_M)
            if sum_A_1_1 == 0:
                if i not in dict_uniq_rows:
                    print(i)
                    dict_uniq_rows[i] = 0
                sum_A_1_1 = 1

            sum_A_1_2 = sum_all_elements_in_row(A_1_2, i, dict_M)
            sum_A_1_3 = sum_all_elements_in_row(A_1_3, i, dict_M)
            cur_value = 0

            if sum_A_1_2 == 0 and sum_A_1_3 == 0:
                cur_value = A_1_1[i, j] / sum_A_1_1
            elif sum_A_1_2 != 0 and sum_A_1_3 == 0:
                cur_value = (1 - gama_1_2) * A_1_1[i, j] / sum_A_1_1
            elif sum_A_1_2 == 0 and sum_A_1_3 != 0:
                cur_value = (1 - gama_1_3) * A_1_1[i, j] / sum_A_1_1
            elif sum_A_1_2 != 0 and sum_A_1_3 != 0:
                cur_value = (1 - gama_1_2 - gama_1_3) * A_1_1[i, j] / sum_A_1_1

            if math.isnan(cur_value):
                print(sum_A_1_1)
                print(sum_A_1_2)
                print(sum_A_1_3)
                print(cur_value)

            M[i, j] = cur_value
            
    return M

In [332]:
def create_inter_transition_matrix(A_1_2, gama_1_2):
    M = np.matrix(np.zeros(A_1_2.shape), dtype=float)
    dict_M = {}
    dict_uniq_rows = {}

    for i in range(M.shape[0]):
        for j in range(M.shape[1]):
            sum_A_1_2 = sum_all_elements_in_row(A_1_2, i, dict_M)
            
            if sum_A_1_2 != 0:
                M[i, j] = gama_1_2 * get_element(A_1_2, i, j) / sum_A_1_2   
            
    return M

In [333]:
M_D_D = create_intra_transition_matrix(A_D_D, A_D_R, A_D_T, gama_drug_disease, gama_target_disease)

2738
2740


In [334]:
M_R_R = create_intra_transition_matrix(A_R_R, A_R_D, A_R_T, gama_drug_disease, gama_target_drug)

1009
1070
1155
1165
1243
1244
1413
1435
1551


In [335]:
M_T_T = create_intra_transition_matrix(A_T_T, A_T_D, A_T_R, gama_target_disease, gama_target_drug)

1607


In [336]:
M_D_R = create_inter_transition_matrix(A_D_R, gama_drug_disease)
M_D_T = create_inter_transition_matrix(A_D_T, gama_target_disease)
M_R_T = create_inter_transition_matrix(A_R_T, gama_target_drug)

In [337]:
M_R_D = M_D_R.transpose()
M_T_D = M_D_T.transpose()
M_T_R = M_R_T.transpose()

In [338]:
M = np.vstack((np.hstack((M_R_R, M_R_T, M_R_D)), np.hstack((M_T_R, M_T_T, M_T_D)), np.hstack((M_D_R, M_D_T, M_D_D))))

assert M.shape[0] == M_R_R.shape[0] + M_T_T.shape[0] + M_D_D.shape[0]
assert M.shape[1] == M_R_R.shape[1] + M_T_T.shape[1] + M_D_D.shape[1]

In [339]:
"""for i in range(M.shape[0]):
    suma = sum_all_elements_in_row(M, i, {})
    for j in range(M.shape[1]):
        if suma != 0:
            M[i, j] = M[i, j] / suma
        else:
            M[i, j] = 1 / M.shape[1]"""

'for i in range(M.shape[0]):\n    suma = sum_all_elements_in_row(M, i, {})\n    for j in range(M.shape[1]):\n        if suma != 0:\n            M[i, j] = M[i, j] / suma\n        else:\n            M[i, j] = 1 / M.shape[1]'

In [340]:
a = np.matrix([2, 3])
b = np.matrix([3, 4])

np.hstack((a, b))

matrix([[2, 3, 3, 4]])

In [341]:
P_R = np.zeros(M_R_R.shape[0])

for i in range(len(P_R)):
    P_R[i] = 1 / len(P_R)

sum(P_R)

0.9999999999999967

In [342]:
P_T = np.zeros(M_T_T.shape[0])

for i in range(len(P_T)):
    P_T[i] = 1 / (len(P_T) + 1)

sum(P_T)

0.9993784959601935

In [403]:
P_D = np.zeros(M_D_D.shape[0])

disease_num = convert_disease_name_to_desease_number("Rheumatoid arthritis, susceptibility to") #Atherosclerosis, susceptibility to #Prostate cancer #Rheumatoid arthritis, susceptibility to

print(disease_num)
P_D[diseases_nodes_list.index(str(int(disease_num)))] = 1.0

print(diseases_nodes_list.index(str(int(disease_num))))

sum(P_D)

180300.0
37


1.0

In [404]:
P_0 = np.matrix(np.concatenate((gama_drug * P_R, gama_target * P_T, gama_disease * P_D))).transpose()
P_t = P_0
sum(P_0)

matrix([[0.9998757]])

In [405]:
count_steps = 0
Zacetek = True

gama = 0.3

while Zacetek or sum(abs(P_t - P_t_prej))[0, 0] > pow(10.0, -8):
    P_t_prej = P_t
    P_t = (1 - gama) * M.transpose() * P_t + gama * P_0
    #print(sum(P_t))
    Zacetek = False
    count_steps += 1
    print(sum(abs(P_t - P_t_prej))[0, 0])
    
count_steps


0.6007410465422669
0.13528498139154035
0.04880060179466926
0.035860208820872756
0.016940796124352446
0.017101678996621877
0.008020629596171915
0.009320750305549788
0.004129469785857984
0.005324885193592458
0.0022106500041710665
0.003144885395485255
0.0012967346862377592
0.0019235643129010768
0.000904181463381359
0.0012092386895251864
0.0006531138156850122
0.0007694668160077543
0.00046056556686251295
0.0004945795555904654
0.0003195662203336577
0.0003204116214515142
0.0002192736235261479
0.00020889033379461964
0.00014927502895756652
0.0001368631436128783
0.00010104689321403914
9.00197681984995e-05
6.811806481749008e-05
5.9387031111790924e-05
4.578050918872883e-05
3.9268728545699766e-05
3.06987860110294e-05
2.6011693087279415e-05
2.055108079710304e-05
1.725341382736566e-05
1.3740593133091113e-05
1.1455818846764887e-05
9.178469000619983e-06
7.612280203809297e-06
6.126755976299357e-06
5.061266346699003e-06
4.087543112012796e-06
3.3666443893780914e-06
2.7259777049478348e-06
2.240173895807187

73

In [406]:
find_n_most_promising = 5
cur_top_values_array = []
cur_top_indexes_array = []

for i in range(find_n_most_promising):
    cur_top_values_array.append(0)
    cur_top_indexes_array.append(0)

for i in range(len(P_R)):
    
    if any(P_t[i] > el for el in cur_top_values_array):
        cur_top_values_array[cur_top_values_array.index(min(cur_top_values_array))] = P_t[i]
        cur_top_indexes_array[cur_top_values_array.index(min(cur_top_values_array))] = i
        
for i in range(find_n_most_promising):
    print("drug: " + convert_drug_id_to_drug_name(drugs_nodes_list[cur_top_indexes_array[i]]) + ", with index: ", cur_top_indexes_array[i], "with value:", cur_top_values_array[i])

drug: Bromhexine, with index:  1549 with value: [[0.00098759]]
drug: Dihydrocodeine, with index:  1224 with value: [[0.00079392]]
drug: Lumefantrine, with index:  1402 with value: [[0.00091831]]
drug: Cytisine, with index:  1551 with value: [[0.00074734]]
drug: Bisacodyl, with index:  1550 with value: [[0.00110564]]


In [394]:
sum_all_elements_in_row(M, 2, {})

0.5999999999999999

In [356]:


rows, cols = (5, 5)
arr = [[0]*cols]*rows
np.matrix(arr) * np.matrix([0, 0, 0, 0, 0]).transpose()

matrix([[0],
        [0],
        [0],
        [0],
        [0]])

Prostate cancer

drug: Capecitabine, with index:  950 with value: [[0.00424314]]
drug: Bicalutamide, with index:  976 with value: [[0.00414983]]
drug: Docetaxel, with index:  1092 with value: [[0.00404751]]
drug: Estramustine, with index:  1041 with value: [[0.00411872]]
drug: Chlordiazepoxide, with index:  347 with value: [[0.00405137]]

Rheumatoid arthritis

drug: Dihydrocodeine, with index:  1224 with value: [[0.00058445]]
drug: Pantoprazole, with index:  99 with value: [[0.000617]]
drug: Temozolomide, with index:  712 with value: [[0.00055556]]
drug: Lumefantrine, with index:  1402 with value: [[0.00062514]]
drug: Estradiol valerate/Dienogest, with index:  1474 with value: [[0.0005551]]

Atherosclerosis

drug: Bromhexine, with index:  1549 with value: [[0.0009875]]
drug: Dihydrocodeine, with index:  1224 with value: [[0.00079384]]
drug: Lumefantrine, with index:  1402 with value: [[0.00091822]]
drug: Bisacodyl, with index:  1550 with value: [[0.00110555]]
drug: Cytisine, with index:  1551 with value: [[0.00074728]]